# 4. Back-Propagation & Autograd(역전파 & 미분 자동화)
***

## 4.1 Computational Graph & Chain Rule(연산 그래프와 연쇄 법칙)

### 4.1.1 Computatonal Graph(연산 그래프)
- 간단한 네트워크에선 경사(gradient)를 구하기 쉬움
    - 단순히 미분하면 됨
- 하지만 복잡한 네트워크라면? 
    - 미분하기가 어려움
 ![4_BP](slides/이미지/4_BP.jpg)

### 4.1.2 Chain Rule(연쇄 법칙)
-  Chain Rule(연쇄법칙): 합성함수의 미분법 
    - 합성함수의 미분은 합성함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있음
    - $f=f(g)$ and $g=g(x)$, ${\partial f \over \partial x} = {\partial f \over \partial g}*{\partial g \over \partial x}$
- Chain Rule을 이용하면 복잡한 네트워크의 경사(Gradient)를 구할 수 있음 
    - 각 과정들의 편미분값을 구한후, 해당 값들을 모두 곱해주면 네트워크의 경사(Gradient)가 나옴
#### <순서>

    1. Local Gradient 구하기
    2. 주어진 Global Gradient 값과 Local Gradient 곱하기

### 4.1.3 Backward Propagation(역전파)
1. Forward pass(전파) 실시
    - 변수($x$값, $y$값), 가중치($w$값)을 모델에 input값으로 넣어 가동
    - 손실(loss) 계산
2. Backward Propagation(역전파) 실시
    - 각각 Gate(node, neuron)의 local gradient 계산(뒤에서부터)
    - 계산시 연쇄 법칙(Chain Rule) 사용
    - 최종 모델의 경사(Gradient) 계산 ($\partial loss \over \partial w$)
3. 가중치 업데이트 실시
    - 계산된 모델의 경사를 가중치에 적용하여 가중치를 조정
    - 조정된 가중치로 모델 훈련 재가동

## 4.2 Code Practice: Backward Propagation & Autograd(역전파 & 미분 자동화)

#### 파이토치에서, 역전파를 진행하기 위해선 변수들만 설정해주면 된다

### 1. 기본 데이터 생성 & 가중치 텐서 설정

In [2]:
import torch
from torch.autograd import Variable # Variable: 파이토치에서 역전파시 사용하는 함수

x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]

w = Variable(torch.Tensor([1.0]), requires_grad = True) # Any random value

# 1. 텐서 생성 2. Variable 함수 사용 3. 경사(gradient) 계산 필요 여부 설정 

### 2. 모델 및 손실 함수 정의

In [3]:
# 1. our forward pass model 
def forward(x):
    return x * w

# 2. Loss function
def loss(x, y):
    y_pred = forward(x)
    return (y_pred - y) * (y_pred - y)

# Before training
print("predict (before training)", 4, forward(4).data[0])

predict (before training) 4 tensor(4.)


### 3. 훈련: 전파, 역전파 그리고 가중치 업데이트

In [4]:
# Training loop
for epoch in range(100):
    for x_val, y_val in zip(x_data, y_data):
        l = loss(x_val, y_val) # 손실 정의
        l.backward() # backward(): 역전파를 통해 손실 계산 
        print("\tgrad: ", x_val, y_val, w.grad.data[0])
        w.data = w.data - 0.01 * w.grad.data # 가중치 업데이트 # 계산된 경사도는 w.grad.data에 저장됨

        # Manually zero the gradients after updating weights
        w.grad.data.zero_()
    print("progress:", epoch, "w=", w, "loss=", l.data[0])

# After training
print("predict(after training)", 4, forward(4).data[0])

	grad:  1.0 2.0 tensor(-2.)
	grad:  2.0 4.0 tensor(-7.8400)
	grad:  3.0 6.0 tensor(-16.2288)
progress: 0 w= tensor([1.2607], requires_grad=True) loss= tensor(7.3159)
	grad:  1.0 2.0 tensor(-1.4786)
	grad:  2.0 4.0 tensor(-5.7962)
	grad:  3.0 6.0 tensor(-11.9981)
progress: 1 w= tensor([1.4534], requires_grad=True) loss= tensor(3.9988)
	grad:  1.0 2.0 tensor(-1.0932)
	grad:  2.0 4.0 tensor(-4.2852)
	grad:  3.0 6.0 tensor(-8.8704)
progress: 2 w= tensor([1.5959], requires_grad=True) loss= tensor(2.1857)
	grad:  1.0 2.0 tensor(-0.8082)
	grad:  2.0 4.0 tensor(-3.1681)
	grad:  3.0 6.0 tensor(-6.5580)
progress: 3 w= tensor([1.7012], requires_grad=True) loss= tensor(1.1946)
	grad:  1.0 2.0 tensor(-0.5975)
	grad:  2.0 4.0 tensor(-2.3422)
	grad:  3.0 6.0 tensor(-4.8484)
progress: 4 w= tensor([1.7791], requires_grad=True) loss= tensor(0.6530)
	grad:  1.0 2.0 tensor(-0.4417)
	grad:  2.0 4.0 tensor(-1.7316)
	grad:  3.0 6.0 tensor(-3.5845)
progress: 5 w= tensor([1.8367], requires_grad=True) loss= ten

	grad:  2.0 4.0 tensor(-2.8610e-06)
	grad:  3.0 6.0 tensor(-5.7220e-06)
progress: 66 w= tensor([2.0000], requires_grad=True) loss= tensor(9.0949e-13)
	grad:  1.0 2.0 tensor(-7.1526e-07)
	grad:  2.0 4.0 tensor(-2.8610e-06)
	grad:  3.0 6.0 tensor(-5.7220e-06)
progress: 67 w= tensor([2.0000], requires_grad=True) loss= tensor(9.0949e-13)
	grad:  1.0 2.0 tensor(-7.1526e-07)
	grad:  2.0 4.0 tensor(-2.8610e-06)
	grad:  3.0 6.0 tensor(-5.7220e-06)
progress: 68 w= tensor([2.0000], requires_grad=True) loss= tensor(9.0949e-13)
	grad:  1.0 2.0 tensor(-7.1526e-07)
	grad:  2.0 4.0 tensor(-2.8610e-06)
	grad:  3.0 6.0 tensor(-5.7220e-06)
progress: 69 w= tensor([2.0000], requires_grad=True) loss= tensor(9.0949e-13)
	grad:  1.0 2.0 tensor(-7.1526e-07)
	grad:  2.0 4.0 tensor(-2.8610e-06)
	grad:  3.0 6.0 tensor(-5.7220e-06)
progress: 70 w= tensor([2.0000], requires_grad=True) loss= tensor(9.0949e-13)
	grad:  1.0 2.0 tensor(-7.1526e-07)
	grad:  2.0 4.0 tensor(-2.8610e-06)
	grad:  3.0 6.0 tensor(-5.7220e-06

#### 파이토치의 autograd를 사용한 것과 직접 함수 모델을 제작해서 사용한 것과 결과는 같음

### 4.2.2 Exercise_1: NumPy를 사용하여 역전파와 계산 그래프를 응용해라

#### 1. 가중치, 모델, 손실 함수 설정

In [174]:
# 0. weight
w = Variable(torch.Tensor([1.0]), requires_grad = True) # Any random value

# 1. our forward pass model 
def forward(x):
    return x * w

# 2. Loss function
def loss(x, y):
    y_pred = forward(x)
    return (y_pred - y) * (y_pred - y)

#### 2. 경사 계산

In [175]:
loss(2, 4).backward() # backward(): 역전파를 통해 손실 계산 
print(w.grad)

tensor([-8.])


### 4.2.3 Exercise_2: 계산 그래프를 이용하여 경사(Gradient)를 계산하라

#### 1. 기본 데이터 & 가중치 텐서 설정

In [178]:
x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]

w_1 = Variable(torch.Tensor([1.0]), requires_grad = True) # Any random value
w_2 = Variable(torch.Tensor([1.0]), requires_grad = True) # Any random value

# 1. 텐서 생성 2. Variable 함수 사용 3. 경사(gradient) 계산 필요 여부 설정 

#### 2. 모델 & 손실 함수 정의

In [184]:
# 0. error term 설정

b = 0

# 1. our forward pass model 
def forward(x,b):
    return (x * x * w_2) + (x * w_1) + b

# 2. Loss function
def loss(x, b, y):
    y_pred = forward(x, b)
    return (y_pred - y) * (y_pred - y)

# Before training
print("predict (before training)", 4, forward(4, 0).data[0])

predict (before training) 4 tensor(20.)


#### 3. 훈련: 전파, 역전파, 가중치 업데이트

In [185]:
# Training loop
for epoch in range(100):
    for x_val, y_val in zip(x_data, y_data):
        l = loss(x_val, b, y_val) # 손실 정의
        l.backward() # backward(): 역전파를 통해 손실 계산 
        print("\tgrad: ", x_val, y_val, w_1.grad.data[0], w_2.grad.data[0])
        w_1.data = w_1.data - 0.01 * w_1.grad.data # 가중치 업데이트 # 계산된 경사도는 w_1.grad.data에 저장됨
        w_2.data = w_2.data - 0.01 * w_2.grad.data # 가중치 업데이트 # 계산된 경사도는 w_2.grad.data에 저장됨

        # Manually zero the gradients after updating weights
        w_1.grad.data.zero_()
        w_2.grad.data.zero_()
    print("progress:", epoch, "w_1=", w_1, "w_2=", w_2, "loss=", l.data[0])

# After training
print("predict(after training)", 4, forward(4, 0).data[0])

	grad:  1.0 2.0 tensor(0.) tensor(0.)
	grad:  2.0 4.0 tensor(8.) tensor(16.)
	grad:  3.0 6.0 tensor(25.9200) tensor(77.7600)
progress: 0 w_1= tensor([0.6608], requires_grad=True) w_2= tensor([0.0624], requires_grad=True) loss= tensor(18.6624)
	grad:  1.0 2.0 tensor(-2.5536) tensor(-2.5536)
	grad:  2.0 4.0 tensor(-9.1023) tensor(-18.2047)
	grad:  3.0 6.0 tensor(-7.4285) tensor(-22.2854)
progress: 1 w_1= tensor([0.8516], requires_grad=True) w_2= tensor([0.4928], requires_grad=True) loss= tensor(1.5328)
	grad:  1.0 2.0 tensor(-1.3110) tensor(-1.3110)
	grad:  2.0 4.0 tensor(-0.9868) tensor(-1.9736)
	grad:  3.0 6.0 tensor(8.1301) tensor(24.3903)
progress: 2 w_1= tensor([0.7933], requires_grad=True) w_2= tensor([0.2818], requires_grad=True) loss= tensor(1.8361)
	grad:  1.0 2.0 tensor(-1.8498) tensor(-1.8498)
	grad:  2.0 4.0 tensor(-4.7010) tensor(-9.4020)
	grad:  3.0 6.0 tensor(0.7510) tensor(2.2531)
progress: 3 w_1= tensor([0.8513], requires_grad=True) w_2= tensor([0.3718], requires_grad=Tr

	grad:  1.0 2.0 tensor(-0.8510) tensor(-0.8510)
	grad:  2.0 4.0 tensor(-1.7830) tensor(-3.5660)
	grad:  3.0 6.0 tensor(1.5760) tensor(4.7279)
progress: 41 w_1= tensor([1.4080], requires_grad=True) w_2= tensor([0.1740], requires_grad=True) loss= tensor(0.0690)
	grad:  1.0 2.0 tensor(-0.8361) tensor(-0.8361)
	grad:  2.0 4.0 tensor(-1.7517) tensor(-3.5034)
	grad:  3.0 6.0 tensor(1.5483) tensor(4.6449)
progress: 42 w_1= tensor([1.4184], requires_grad=True) w_2= tensor([0.1709], requires_grad=True) loss= tensor(0.0666)
	grad:  1.0 2.0 tensor(-0.8214) tensor(-0.8214)
	grad:  2.0 4.0 tensor(-1.7209) tensor(-3.4419)
	grad:  3.0 6.0 tensor(1.5211) tensor(4.5634)
progress: 43 w_1= tensor([1.4286], requires_grad=True) w_2= tensor([0.1679], requires_grad=True) loss= tensor(0.0643)
	grad:  1.0 2.0 tensor(-0.8070) tensor(-0.8070)
	grad:  2.0 4.0 tensor(-1.6907) tensor(-3.3814)
	grad:  3.0 6.0 tensor(1.4944) tensor(4.4833)
progress: 44 w_1= tensor([1.4386], requires_grad=True) w_2= tensor([0.1650], r

	grad:  2.0 4.0 tensor(-0.8325) tensor(-1.6649)
	grad:  3.0 6.0 tensor(0.7358) tensor(2.2074)
progress: 84 w_1= tensor([1.7236], requires_grad=True) w_2= tensor([0.0812], requires_grad=True) loss= tensor(0.0150)
	grad:  1.0 2.0 tensor(-0.3904) tensor(-0.3904)
	grad:  2.0 4.0 tensor(-0.8178) tensor(-1.6357)
	grad:  3.0 6.0 tensor(0.7229) tensor(2.1687)
progress: 85 w_1= tensor([1.7284], requires_grad=True) w_2= tensor([0.0798], requires_grad=True) loss= tensor(0.0145)
	grad:  1.0 2.0 tensor(-0.3835) tensor(-0.3835)
	grad:  2.0 4.0 tensor(-0.8035) tensor(-1.6070)
	grad:  3.0 6.0 tensor(0.7102) tensor(2.1306)
progress: 86 w_1= tensor([1.7332], requires_grad=True) w_2= tensor([0.0784], requires_grad=True) loss= tensor(0.0140)
	grad:  1.0 2.0 tensor(-0.3768) tensor(-0.3768)
	grad:  2.0 4.0 tensor(-0.7894) tensor(-1.5788)
	grad:  3.0 6.0 tensor(0.6977) tensor(2.0932)
progress: 87 w_1= tensor([1.7379], requires_grad=True) w_2= tensor([0.0770], requires_grad=True) loss= tensor(0.0135)
	grad:  